In [24]:
import tiktoken
import torch

In [25]:
with open('corpus.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(len(text))

1848542


In [26]:
chars = sorted(list(set(text)))
vocab_size=len(chars)
print(vocab_size)
print(' '.join(chars))

28

   a b c d e f g h i j k l m n o p q r s t u v w x y z


In [27]:
## character level encoding
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[9, 10, 10, 1, 21, 9, 6, 19, 6]
hii there


In [28]:
enc=tiktoken.get_encoding('gpt2')
enc.n_vocab
# enc.encode("hegel")
enc.decode([258, 25280])
##subword encodings

'hegel'

In [29]:
data=torch.tensor(encode(text), dtype=torch.long)
print(data.shape)

torch.Size([1848542])


In [30]:
n = int(0.8*len(data))
train_data=data[:n]
test_data=data[n:]

In [31]:
context_length = 16
train_data[:context_length+1]

tensor([ 0, 17, 19,  6,  7,  2,  4,  6,  1,  2, 13, 21,  9, 16, 22,  8,  9])

In [32]:
x = train_data[:context_length]
y = train_data[1:context_length+1]
for t in range(context_length):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

## use context sizes from 1 to context_length because
# 1. computationally better
# 2. transformer can work on generating with as less as 1 token input
# if greater than context size truncation is required
#all this will be done in batches processed parallely at same time 

when input is tensor([0]) the target: 17
when input is tensor([ 0, 17]) the target: 19
when input is tensor([ 0, 17, 19]) the target: 6
when input is tensor([ 0, 17, 19,  6]) the target: 7
when input is tensor([ 0, 17, 19,  6,  7]) the target: 2
when input is tensor([ 0, 17, 19,  6,  7,  2]) the target: 4
when input is tensor([ 0, 17, 19,  6,  7,  2,  4]) the target: 6
when input is tensor([ 0, 17, 19,  6,  7,  2,  4,  6]) the target: 1
when input is tensor([ 0, 17, 19,  6,  7,  2,  4,  6,  1]) the target: 2
when input is tensor([ 0, 17, 19,  6,  7,  2,  4,  6,  1,  2]) the target: 13
when input is tensor([ 0, 17, 19,  6,  7,  2,  4,  6,  1,  2, 13]) the target: 21
when input is tensor([ 0, 17, 19,  6,  7,  2,  4,  6,  1,  2, 13, 21]) the target: 9
when input is tensor([ 0, 17, 19,  6,  7,  2,  4,  6,  1,  2, 13, 21,  9]) the target: 16
when input is tensor([ 0, 17, 19,  6,  7,  2,  4,  6,  1,  2, 13, 21,  9, 16]) the target: 22
when input is tensor([ 0, 17, 19,  6,  7,  2,  4,  6,  1,

In [33]:
batch_size = 4 # how many independent sequences will we process in parallel?
context_length = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - context_length, (batch_size,))
    x = torch.stack([data[i:i+context_length] for i in ix])
    y = torch.stack([data[i+1:i+context_length+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(context_length): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[22, 19,  6,  1, 19,  6,  7, 13],
        [ 1, 20,  6, 13,  7,  1, 12, 15],
        [21, 13, 26,  1, 17, 19,  6, 20],
        [20, 21, 10,  4,  1, 19,  6,  2]])
targets:
torch.Size([4, 8])
tensor([[19,  6,  1, 19,  6,  7, 13,  6],
        [20,  6, 13,  7,  1, 12, 15, 16],
        [13, 26,  1, 17, 19,  6, 20,  6],
        [21, 10,  4,  1, 19,  6,  2, 13]])
----
when input is [22] the target: 19
when input is [22, 19] the target: 6
when input is [22, 19, 6] the target: 1
when input is [22, 19, 6, 1] the target: 19
when input is [22, 19, 6, 1, 19] the target: 6
when input is [22, 19, 6, 1, 19, 6] the target: 7
when input is [22, 19, 6, 1, 19, 6, 7] the target: 13
when input is [22, 19, 6, 1, 19, 6, 7, 13] the target: 6
when input is [1] the target: 20
when input is [1, 20] the target: 6
when input is [1, 20, 6] the target: 13
when input is [1, 20, 6, 13] the target: 7
when input is [1, 20, 6, 13, 7] the target: 1
when input is [1, 20, 6, 13, 7, 1] the ta

In [35]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 28])
tensor(4.2440, grad_fn=<NllLossBackward0>)

en u
nqkpiqhvtybnsvnfkomoty
ourqytygmv jkqowbwpbxdzhuleomhtq wiu
 bcpkqhae ycpwahy
haqcscjnsguxjfrqy


In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())
